# HW07: Deep Learning (due 16th November)

In this homework, you will replicate the heterogenous controlment effect exercise from last week's homework using a deep learning model instead of a machine learning one. 

For those who did not complete this exercise, you will investigate the effect of case management on mental health outcomes. These data come from a randomized control trial where patients were assigned to *intensive* or *standard* case management. In this context, the controlment is being assigned to the **intensive** case management while patients assigned to the **standard** case management belong to the control group.

We will investigate characteristics of individuals who are most and least responsive to the controlment, i.e., to being assigned to the intensive case management. 

In [44]:
import pandas as pd
from tensorflow import keras

In [45]:
import pandas as pd
import statsmodels.formula.api as smf

df = pd.read_stata('http://www.homepages.ucl.ac.uk/~rmjwiww/stata/missing/uk500.dta')
df = df.dropna()
pd.read_stata('http://www.homepages.ucl.ac.uk/~rmjwiww/stata/missing/uk500.dta', iterator=True).variable_labels()

{'trialid': 'Trial ID',
 'centreid': 'Trial centre',
 'status': 'Patient status at baseline',
 'age': 'Age in years at baseline',
 'sex': 'Sex',
 'afcarib': 'Ethnic group',
 'ocfabth': "Father's social class at birth",
 'chron1l': 'Months since onset of psychosis, logged',
 'hos94': 'Days in hospital for psychiatric reasons: 2 years before baseline',
 'cprs94': 'Psychopathology at baseline (CPRS)',
 'das94': 'Disability at baseline (DAS)',
 'sat94': '(Dis)satisfaction with services at baseline',
 'rand': 'Randomised group',
 'hos96': 'Days in hospital for psychiatric reasons: 2 years after baseline',
 'cprs96': 'Psychopathology at 2 years (CPRS)',
 'sat96': '(Dis)satisfaction with services at 2 years'}

In [46]:
df.head()

,trialid,centreid,status,age,sex,afcarib,ocfabth,chron1l,hos94,cprs94,das94,sat94,rand,hos96,cprs96,sat96
1,107.0,St George's,Out-patient,27.0,male,Other,A,3.178054,80.0,4.0,0.285714,18.0,Intensive case management,27.0,3.0,22.000
2,222005.0,St Mary's,In hospital,41.0,male,Other,D,4.521789,240.0,6.0,0.750000,15.0,Intensive case management,15.0,13.0,9.000
3,222018.0,St Mary's,In hospital,25.0,male,Other,C2,4.094345,48.0,12.0,0.125000,18.0,Intensive case management,263.0,6.0,21.375
5,312015.0,King's,Out-patient,31.0,female,Other,A,4.787492,60.0,28.0,2.375000,20.0,Intensive case management,45.0,19.0,17.000
6,221023.0,St Mary's,In hospital,35.0,male,Afro-Caribbean,C2,4.430817,60.0,25.0,1.571428,24.0,Intensive case management,58.0,27.0,19.125


The treatment variable is $rand$, the post-treatment outcomes are $hos96$, $cprs96$ and $sat96$.

In [47]:
treatvar = 'rand'
df[treatvar].value_counts()

Intensive case management    130
Standard case management     116
Name: rand, dtype: int64

In [48]:
outcomes = ['sat96', 'hos96', 'cprs96']
df[outcomes].describe()

,sat96,hos96,cprs96
count,246.000000,246.000000,246.000000
mean,17.271341,65.500000,17.790587
std,4.723009,104.046722,14.090911
min,9.000000,0.000000,0.000000
25%,14.000000,0.000000,7.000000
50%,17.000000,15.000000,15.000000
75%,20.187500,93.500000,26.000000
max,32.000000,692.000000,71.000000


In addition to these variables we need a set of covariates that we want to use to identify individuals who are most and least responsive to treatment. We also encode categorical covariates and prepare them for the ML model.

In [49]:
#Encoding Categorical covariates and preparing the data for tensorflow
covariates = ['status', 'sex', 'sat94', 'ocfabth', 'hos94', 'das94', 'cprs94', 'age', 'afcarib']
covariates_cat = ['status', 'sex', 'ocfabth', 'afcarib']

from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
df[covariates_cat] = encoder.fit_transform(df[covariates_cat])
df[covariates] = df[covariates].astype('float32')
df[covariates].head()

,status,sex,sat94,ocfabth,hos94,das94,cprs94,age,afcarib
1,1.0,1.0,18.0,0.0,80.0,0.285714,4.0,27.0,1.0
2,0.0,1.0,15.0,4.0,240.0,0.750000,6.0,41.0,1.0
3,0.0,1.0,18.0,3.0,48.0,0.125000,12.0,25.0,1.0
5,1.0,0.0,20.0,0.0,60.0,2.375000,28.0,31.0,1.0
6,0.0,1.0,24.0,3.0,60.0,1.571428,25.0,35.0,0.0


In [50]:
# Subset the dataset by treatment and control
# Within each sample, create a training, a test and a validation set

df_treat = df.loc[df[treatvar]=='Intensive case management']
df_control = df.loc[df[treatvar]!='Intensive case management']

Xt_train, Xt_test, yt_train, yt_test = train_test_split(df_treat[covariates], df_treat['hos96'], test_size=0.3)
Xc_train, Xc_test, yc_train, yc_test = train_test_split(df_control[covariates], df_control['hos96'], test_size=0.3)

In [51]:
##choose one of the three outcomes to analyze
## build two MLP (multilayer perceptron) models to predict the outcome based on the covariates
## the first model should be trained on the treated sample, while the second on the control

# the two MLP models should have with at least 2 hidden layers, ReLU activation, batch normalization, dropout

from tensorflow import keras

model_treat = keras.models.Sequential()

model_treat.add(keras.layers.Dense(50, input_dim = Xt_train.shape[1], activation="relu"))
model_treat.add(keras.layers.BatchNormalization())
keras.layers.Dropout(0.3)

model_treat.add(keras.layers.Dense(30, activation="relu"))
model_treat.add(keras.layers.BatchNormalization())
keras.layers.Dropout(0.3)

model_treat.add(keras.layers.Dense(1))

model_treat.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 50)                500       
                                                                 
 batch_normalization_14 (Bat  (None, 50)               200       
 chNormalization)                                                
                                                                 
 dense_22 (Dense)            (None, 30)                1530      
                                                                 
 batch_normalization_15 (Bat  (None, 30)               120       
 chNormalization)                                                
                                                                 
 dense_23 (Dense)            (None, 1)                 31        
                                                                 
Total params: 2,381
Trainable params: 2,221
Non-traina

In [52]:
model_control = keras.models.Sequential()

model_control.add(keras.layers.Dense(50, input_dim = Xt_train.shape[1], activation="relu"))
model_control.add(keras.layers.BatchNormalization())
keras.layers.Dropout(0.3)

model_control.add(keras.layers.Dense(30, activation="relu"))
model_control.add(keras.layers.BatchNormalization())
keras.layers.Dropout(0.3)

model_control.add(keras.layers.Dense(1))

In [53]:
# compile the models

model_treat.compile(loss="mean_squared_error",
              optimizer="adam",
              metrics=["mean_squared_error"])

model_control.compile(loss="mean_squared_error",
              optimizer="adam",
              metrics=["mean_squared_error"])

In [54]:
# fit separate models on the controlment dataset and control dataset
# use early stopping
es = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

print("Treatment model")
res_treat = model_treat.fit(Xt_train, yt_train, epochs=8, validation_split=0.2, callbacks=[es])
print("")
print("Control model")
res_control = model_control.fit(Xc_train, yc_train, epochs=8, validation_split=0.2, callbacks=[es])

Treatment model
Epoch 1/8
3/3 [==============================] - 1s 99ms/step - loss: 12772.8174 - mean_squared_error: 12772.8174 - val_loss: 6642.1528 - val_mean_squared_error: 6642.1528
Epoch 2/8
3/3 [==============================] - 0s 16ms/step - loss: 12744.9834 - mean_squared_error: 12744.9834 - val_loss: 6536.2441 - val_mean_squared_error: 6536.2441
Epoch 3/8
3/3 [==============================] - 0s 16ms/step - loss: 12748.2383 - mean_squared_error: 12748.2383 - val_loss: 6499.8115 - val_mean_squared_error: 6499.8115
Epoch 4/8
3/3 [==============================] - 0s 15ms/step - loss: 12721.1025 - mean_squared_error: 12721.1025 - val_loss: 6478.4102 - val_mean_squared_error: 6478.4102
Epoch 5/8
3/3 [==============================] - 0s 16ms/step - loss: 12700.6201 - mean_squared_error: 12700.6201 - val_loss: 6468.4287 - val_mean_squared_error: 6468.4287
Epoch 6/8
3/3 [==============================] - 0s 15ms/step - loss: 12670.3320 - mean_squared_error: 12670.3320 - val_loss

In [55]:
##get predicted outcomes using the combined test sets for both models 

test_set = Xt_test.append(Xc_test) 
yhat_treat = model_treat.predict(test_set)
yhat_control = model_control.predict(test_set)

3/3 [==============================] - 0s 2ms/step


C:\Users\kschuepbach\AppData\Local\Temp\ipykernel_13164\184693369.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_set = Xt_test.append(Xc_test)


3/3 [==============================] - 0s 2ms/step


In [56]:
##find the three individuals in the test set that are most and least responsive to the controlment
##namely the three individuals for who the controlment effect is larger and those for who it is smaller

#find the three individuals in the test set that are most and least responsive to the treatment
##namely the three individuals for who the treatment effect is larger and those for who it is smaller

test_set['treat_effect'] = yhat_treat - yhat_control
test_set = test_set.sort_values('treat_effect', axis=0)
most_least_treat = test_set[:3].append(test_set[-3:])
most_least_treat

C:\Users\kschuepbach\AppData\Local\Temp\ipykernel_13164\929883172.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  most_least_treat = test_set[:3].append(test_set[-3:])


,status,sex,sat94,ocfabth,hos94,das94,cprs94,age,afcarib,treat_effect
327,1.0,0.0,23.0,0.0,68.0,4.714283,67.0,47.0,0.0,-1.361142
494,0.0,1.0,20.0,1.0,42.0,1.000000,39.0,30.0,1.0,-0.672266
427,0.0,1.0,22.0,1.0,40.0,1.142857,30.0,24.0,1.0,-0.105249
126,1.0,1.0,17.0,4.0,300.0,0.428571,29.0,41.0,1.0,9.725849
260,0.0,1.0,23.0,4.0,330.0,1.500000,17.0,47.0,1.0,11.035285
480,0.0,1.0,18.0,3.0,420.0,1.799999,30.0,33.0,0.0,13.378606


In [57]:
##visualize and comment on the covariates of these individuals 

# I'm not sure how to visualize the covariates, I thought it is not straightforward to interpret the covariates in a neural net?